**Upload "ddi_biotech 022422 Format.csv" and "DrugList.txt" before start.**

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

2023-09-27 17:37:33.638802: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 17:37:35.712524: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-27 17:37:38.413444: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-27 17:37:38.414128: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [ ]:
import numpy as np
import pandas as pd
import spacy

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_interaction = pd.read_csv('/content/drive/MyDrive/mediAI/DDI-CNN/data/ddi_biotech 022422 Format.csv')
df_interaction

,drugbank_id1,drug1,drugbank_id2,drug2,interaction
0,DB00001,Lepirudin,DB06605,Apixaban,Apixaban may increase the anticoagulant activi...
1,DB00001,Lepirudin,DB06695,Dabigatran etexilate,Dabigatran etexilate may increase the anticoag...
2,DB00001,Lepirudin,DB01254,Dasatinib,The risk or severity of bleeding and hemorrhag...
3,DB00001,Lepirudin,DB01609,Deferasirox,The risk or severity of gastrointestinal bleed...
4,DB00001,Lepirudin,DB01586,Ursodeoxycholic acid,The risk or severity of bleeding and bruising ...
...,...,...,...,...,...
249986,DB16746,Elivaldogene autotemcel,DB01264,Darunavir,The therapeutic efficacy of Elivaldogene autot...
249987,DB16746,Elivaldogene autotemcel,DB01319,Fosamprenavir,The therapeutic efficacy of Elivaldogene autot...
249988,DB16746,Elivaldogene autotemcel,DB01601,Lopinavir,The therapeutic efficacy of Elivaldogene autot...
249989,DB16746,Elivaldogene autotemcel,DB11586,Asunaprevir,The therapeutic efficacy of Elivaldogene autot...


In [ ]:
print(len(df_interaction['drug2']))

249991


In [ ]:
drugList_A = df_interaction.drug1.unique().tolist()
print(f'Number of unique drugA:{len(drugList_A)}')
drugList_B = df_interaction.drug2.unique().tolist()
print(f'Number of unique drugB:{len(drugList_B)}')
drugList_A.extend(drugList_B)
drugList = [*set(drugList_A)]
print(f'Number of unqiue drugA + drugB:{len(drugList)}')

Number of unique drugA:660
Number of unique drugB:3924
Number of unqiue drugA + drugB:3988


In [ ]:
import spacy

# Load English language model for spacy
nlp = spacy.load('en_core_web_sm')

# Define function for NLP processing
def NLPProcess(druglist, df_interaction):
    # Initialize lists to store results
    mechanism=[]
    action=[]
    drugA=[]
    drugB=[]

    # Iterate through each interaction in the input dataframe
    for event in events:
        # Process the text with spacy NLP pipeline
        doc = nlp(event)

        # Initialize lists to store dependency relations and children of each word
        dependency = [word for word in doc]
        sons = [[] for i in range(len(dependency))]
        sonsNum = np.zeros(len(dependency))

        # Identify the root and action of the sentence
        for word in dependency:
            if word.dep_ == 'ROOT':
                action.append(word.lemma_)
                root = int(word.i)

        # Identify the two drugs mentioned in the sentence
        drug_count = 0
        for word in dependency:
            if word.text in druglist:
                if drug_count == 0:
                    drugA.append(word.text)
                    drug_count += 1
                elif drug_count == 1:
                    drugB.append(word.text)
                    drug_count += 1

        # Identify the children of each word and their dependency relations
        for word in dependency:
            sonsNum[int(word.head.i)] += 1
            sons[int(word.head.i)].append(int(word.i))

        # Identify the mechanism of action
        mechanism_nodes = [int(son) for son in sons[root] if dependency[son].dep_ in ['dobj', 'nsubjpass']]
        quene = mechanism_nodes[:]
        for node in mechanism_nodes:
            quene += [int(son) for son in sons[node] if dependency[son].text not in drugA[-1:] + drugB[-1:]]
        mechanism_nodes += quene
        mechanism_nodes.sort()
        mechanism.append(" ".join([dependency[node].text for node in mechanism_nodes]))

        # Perform simple text substitutions to improve readability
        if mechanism[-1] == "the fluid retaining activities":
            mechanism[-1] = "the fluid"
        if mechanism[-1] == "atrioventricular blocking ( AV block )":
            mechanism[-1] = "the atrioventricular blocking ( AV block ) activities increase"

    return mechanism, action, drugA, drugB

# Read input dataframe
#df_interaction = pd.read_csv("input_data.csv")
events = df_interaction['interaction']

# Read drug list
#drugList = []
#with open("DrugList.txt", 'r') as f:
#    for line in f:
#        drugList.append(line.strip())
drugList_A = df_interaction.drug1.unique().tolist()
drugList_B = df_interaction.drug2.unique().tolist()
drugList_A.extend(drugList_B)
drugList = [*set(drugList_A)]

# Process NLP on input data
mechanism, action, drugA, drugB = NLPProcess(drugList, df_interaction)

# Ensure all lists are of the same length
min_length = min(len(mechanism), len(action), len(drugA), len(drugB))
mechanism = mechanism[:min_length]
action = action[:min_length]
drugA = drugA[:min_length]
drugB = drugB[:min_length]


In [ ]:
# Create output dataframe
d = {'mechanism': mechanism, 'action': action, 'drugA': drugA, 'drugB': drugB}
df = pd.DataFrame(data=d)

# Write output dataframe to CSV file
df.to_csv("output_data.csv", index=False)

In [ ]:
print(len(mechanism))
print(len(action))
print(len(drugA))
print(len(drugB))

141950
141950
141950
141950


In [ ]:
# Create output dataframe
d = {'mechanism': mechanism, 'action': action, 'drugA': drugA, 'drugB': drugB}
df = pd.DataFrame(data=d)

# Write output dataframe to CSV file
df.to_csv("output_data.csv", index=False)
df

,mechanism,action,drugA,drugB
0,the anticoagulant activities activities of,increase,Apixaban,Lepirudin
1,the anticoagulant activities activities of,increase,Dabigatran,Lepirudin
2,The risk risk or severity of,increase,Dasatinib,Lepirudin
3,The risk risk or severity of,increase,Lepirudin,Deferasirox
4,The risk risk or severity of,increase,Lepirudin,Lepirudin
...,...,...,...,...
141945,The risk risk or severity of,increase,Tremelimumab,Amivantamab
141946,The risk risk or severity of,increase,Olaratumab,Amivantamab
141947,The risk risk or severity of,increase,Tremelimumab,Amivantamab
141948,The risk risk or severity of,increase,TB-402,Amivantamab
